In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
vgg_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3), pooling='avg')
# vgg_model.summary()

In [3]:
def features_response(image, model, layers_name=None):
    
    if layers_name is None:
        layers_name = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    
        
    filter_dict = {}
    outputs = [layer.output for layer in model.layers if layer.name in layers_name]
    model = Model(inputs=model.input, outputs=outputs)
    
    feature_maps = model.predict(image)
    for layer, filters in zip(layers_name, feature_maps):
        filter_dict[layer] = filters
            
    return filter_dict

In [4]:
def load_image(path, preprocess=False):
    
    img = image.load_img(path, target_size=(224, 224))
    img = image.img_to_array(img)
    if preprocess:
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
    
    return img

In [5]:
def content_loss(content, combine):
    squared_difference = tf.square(content - combine)
    
    return tf.reduce_mean(squared_difference, axis=-1)

def gram_matrix(feature_map):
    height, width, depth = feature_map.shape

    feature_map = tf.reshape(feature_map, (depth, height * width))
    gram_matrix = tf.matmul(feature_map, tf.transpose(feature_map))
    
    return gram_matrix

def style_loss(style, combination):
    
    width, height, filters = style.shape
    
    style = gram_matrix(style)
    combination = gram_matrix(combination)
    
    layer_loss = tf.square(style - combination)
    
    return 1 / (4 * (width * height) * filters**2 * tf.reduce_mean(layer_loss))

In [48]:
content_image = tf.Variable(load_image('./styles/kitten.jpg', preprocess=True))
style_image = tf.Variable(load_image('./styles/starry_night.jpg', preprocess=True))

In [49]:
style_features = features_response(style, vgg_model)
content_features = features_response(content, vgg_model)

1/1 [==============================] - 0s 133ms/step


In [51]:
combination_image = np.random.randint(0, 255, (content.shape[1:])).astype(np.uint8)
combination_image = tf.Variable(np.expand_dims(combination_image, axis=0))

In [46]:
def step(content, combination):

    with tf.GradientTape() as tape:

        pred = vgg_model(X)
        
        y = content_features['block5_conv1']
        loss = content_loss(y, content)

    grads = tape.gradient(loss, content)


In [47]:
EPOCHS = 25
BS = 64
INIT_LR = 1e-3

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

for epoch in range(0, EPOCHS):
    
    print("[INFO] starting epoch {}/{}...".format(epoch + 1, EPOCHS), end="")
    sys.stdout.flush()
    epochStart = time.time()

    loss, grads = step(x, content)
    opt.apply_gradients(zip(grads, content))
    
    
    epochEnd = time.time()
    elapsed = (epochEnd - epochStart) / 60.0
    print("took {:.4} minutes".format(elapsed))

vgg_model.compile(optimizer=opt, loss=content_loss, metrics=["MSE"])

[INFO] starting epoch 1/25...

InvalidArgumentError: Exception encountered when calling layer "block1_conv1" (type Conv2D).

Value for attr 'T' of uint8 is not in the list of allowed values: half, bfloat16, float, double, int32
	; NodeDef: {{node Conv2D}}; Op<name=Conv2D; signature=input:T, filter:T -> output:T; attr=T:type,allowed=[DT_HALF, DT_BFLOAT16, DT_FLOAT, DT_DOUBLE, DT_INT32]; attr=strides:list(int); attr=use_cudnn_on_gpu:bool,default=true; attr=padding:string,allowed=["SAME", "VALID", "EXPLICIT"]; attr=explicit_paddings:list(int),default=[]; attr=data_format:string,default="NHWC",allowed=["NHWC", "NCHW"]; attr=dilations:list(int),default=[1, 1, 1, 1]> [Op:Conv2D]

Call arguments received by layer "block1_conv1" (type Conv2D):
  • inputs=<tf.Variable 'Variable:0' shape=(1, 224, 224, 3) dtype=uint8, numpy=
array([[[[100, 222,  86],
         [110,  83, 109],
         [124,  59,  35],
         ...,
         [146, 166, 176],
         [163, 202, 220],
         [  8,  79, 137]],

        [[ 48, 237, 191],
       

In [42]:
vgg_model.predict(x)

1/1 [==============================] - 0s 145ms/step


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 